<a href="https://colab.research.google.com/github/TomokiArima/Simulation2024_Homework/blob/main/%E3%82%B7%E3%83%9F%E3%83%A5%E5%B7%A5_%E8%AA%B2%E9%A1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import matplotlib.pyplot as plt
import numpy as np
import math
import ipywidgets as widgets    #スライダー用のライブラリ
from IPython.display import display


#減衰の式
def DampedOscillation (w0,k,x0,wd, t):
  x = [] #結果を格納するリストを初期化
  for ti in t: #tの各要素に対してループ、tiはi番目の配列の中身
    xi = x0 * (math.exp(-k * ti)) *(math.cos(wd * ti) + ((k/wd) * math.sin(wd * ti)))
    x.append(xi) # 結果をリストに追加
  return np.array(x) #リストをNumPy配列に変換して返す


#条件
t = np.arange(0, 10, 0.01)  #時間範囲
x0=7  #初期変位x0
V = None

#スライダーの設定
w0_slider = widgets.FloatSlider(value=5, min=0.1, max=10, step=0.1, description='ω₀:')  #w0：固有角振動数　(value=初期値、min,max=スライダー範囲、step=刻み幅、description=スライダータイトル)
k_slider = widgets.FloatSlider(value=0.1, min=0.01, max=1, step=0.01, description='k:') #k：減衰係数

#グラフを出力する所を作成
output = widgets.Output()

#関数：スライダーの値が変更されたときの更新処理
def update_plot(change):
  global V  #Vをグローバル変数として扱う
  with output:  #グラフを出力する所のoutputに対する処理という指定
    clear_output(wait=True)   #以前の出力を削除する
    #スライダーの値をw0,kに代入する
    w0 = w0_slider.value  #".value"でスライダーの値を取得する
    k = k_slider.value

    try:
      wd = math.sqrt((w0**2) - (k**2))  #wd：角振動数(減衰有)
      V = DampedOscillation(w0,k,x0,wd,t) #V：変位　減衰の式(DampedOscillation)に各値を代入して計算


      plt.plot(t, V)
      plt.xlabel('t')
      plt.ylabel('V')
      plt.title(f'w0 = {w0:.2f}, k = {k:.2f}, x₀ = {x0:.2f}')
      plt.grid(True) # グリッド表示
      plt.show()

    except ValueError:
            print("Error: wd=√(ω₀² - k²)が負になった可能性")

#スライダーの値の変化を検知する
w0_slider.observe(update_plot, names='value')  #names='value'でスライダーの値を検知する指定。　検知された際に呼び出す関数はupdate_plot
k_slider.observe(update_plot, names='value')

display(w0_slider, k_slider, output) #スライダーと出力エリアをNotebook上に表示
update_plot(None)




FloatSlider(value=5.0, description='ω₀:', max=10.0, min=0.1)

FloatSlider(value=0.1, description='k:', max=1.0, min=0.01, step=0.01)

Output()